In [ ]:
import os
import numpy as np
import math
import librosa

In [ ]:
"""
    Script parameters
"""
target_amount_samples = 250
skipSamples = 0

"Persons"

speakers_in_room = 1

"ROOM"
randomize_room = False
# room generation will be laplace distributed in these intervalls [width, length, height]
normal_room_dim = [15, 20, 4]
room_dim_ranges = [[3, 30], [3, 30], [2.5, 5]]

# The amout Walls absorb Sound
normal_absorption = 1.2
absorption_range = [1, 2]

# the time it take until the signal drops by 60 dB
normal_rt60 = 0.5
rt60_range = [0.05, 0, 75]

"AUdio"
sampleRate = 16000
#total_length = 30
max_audio_length = 30
#min_audio_length = 5
min_characters = 20


In [ ]:
def rec_filter(x):
    return True if 'rec' in x else False


def wav_filter(x):
    return True if '.wav' in x else False


def txt_filter(x):
    return True if '.TextGrid' in x else False


def as_filter(x):
    return True if 'AS' in x else False


def idx_filter_txt(x, y):
    return True if x[len(x) - 10] == y else False


def rec_filter(x):
    return True if 'rec' in x else False


def norm2(x):
    return np.sum(map(lambda y: y**2, x))


def distance(x,y):
    return norm2(x-y)

def avg(x):
    return sum(x)/len(x)

In [ ]:
def FileGeneratorsKEC():
    src = '/workspace/data/KEC'
    recFolders = list(filter(lambda f: rec_filter(f), os.listdir(src)))
    allWavfiles = []
    alltxtFiles = []
    for rec in recFolders:
        currentDir = src+"/"+rec

        wavFilesOfFolder = list(filter(lambda f: wav_filter(f), os.listdir(currentDir)))
        wavFilesOfFolder = list(map(lambda f: currentDir+f, wavFilesOfFolder))
        allWavfiles.extend(wavFilesOfFolder)

        txtFilesOfFolder = list(filter(lambda f: txt_filter(f), os.listdir(currentDir)))
        txtFilesOfFolder = list(map(lambda f: currentDir+f, txtFilesOfFolder))
        alltxtFiles.extend(txtFilesOfFolder)
    
    def wavGen():
        for file in allWavfiles:
            yield file
    def textGen():
        for file in alltxtFiles:
            yield file

    return wavGen, textGen

In [ ]:
def generate_room_characteristics():
    dims,rt60,absorption=0
    if randomize_room:
        dims: list(int) = [np.random.randint(room_dim_ranges[i, 0], room_dim_ranges[i, 1])
                           for i in range(len(room_dim_ranges[:]))]

        rt60: float = rt60_range[0] + \
            (rt60_range[1]-rt60_range[0])*np.random.random()

        absorption: float = absorption_range[0] + \
            (absorption_range[1]-absorption_range[0]) * np.random.random()

    else:
        dims = normal_room_dim
        rt60 = normal_rt60
        absorption = normal_absorption

    return dims, rt60, absorption

In [ ]:
def random_position_in_room(roomDims):
    x = np.random.random() * (roomDims[0]-1) + 0.5
    y = np.random.random() * (roomDims[1]-1) + 0.5
    z = 1.73
    return [x, y, z]


def positions_too_close(positions):
    for a in positions[1:]:
        for b in positions[1:]:
            if a == b:
                continue
            if distance(a, b) < 1:
                return True
    return False


# returns angle between two points in degrees
def get_angle(a, b):
    # turns clockwise angle into counter-clockwise
    def angle_trunc(a):
        while a < 0.0:
            a += math.pi * 2
        return a

    deltaY = b[1] - a[1]
    deltaX = b[0] - a[0]
    return math.degrees(angle_trunc(math.atan2(deltaY, deltaX)))


def transform_to_directivities(positions):
    dirs = []
    middle = [avg(positions[:, i]) for i in len(positions[0])]

    posi = positions[0]
    # Winkel aus Vogelperspeltive, in die Listnener guckt
    baseAngle = get_angle(posi[:2],middle[:2])

    # linkes und rechtes ohr des LIsteners
    dirs.append([(baseAngle + 90) % 360, 90])  # bug somewhere here
    dirs.append([(baseAngle + 270) % 360, 90])

    for pos in positions[2:]:
        dirs.append([get_angle(pos,posi), 90])

    return dirs


def random_persons_in_room(roomDims):
    def pos_in_room(count):
        pos = []
        for i in range(count):
            pos.append(random_position_in_room(roomDims))
        return pos

    # do while: erzeuge solange bis gültiges ergebnis
    positions = pos_in_room(speakers_in_room)
    while(positions_too_close(positions)):
        positions = pos_in_room(speakers_in_room)
    positions[0] = positions[1]

    return positions, transform_to_directivities(positions)

In [ ]:
def loadWavFile(path, offset, duration):
    wav = librosa.load(path, sr=sampleRate, offset=offset,
                       duration=duration, mono=True)
    print(wav)

In [ ]:

def generate():
    wavGen, txtGen = FileGeneratorsKEC()
    for wav in wavGen():
        loadWavFile(wav, 0,10)
        break


if __name__ == '__main__':
    generate()
